In [1]:
### BEWARE:Tensorflow is stochastic - this means the model will not be replicated exactly. 
### Use GA_Load_Model for reproduction

#!pip install mlxtend

#!pip install h5py pyyaml

#!pip install tensorboard

#!pip install tensorflow

In [2]:
#Load packages

%load_ext tensorboard

%matplotlib inline

In [3]:
### Packages necessary for model construction 

import tensorflow as tf
import tensorflow.keras.callbacks
import datetime 
import statistics

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os 

from matplotlib import pyplot as plt

import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
#Read the Data

UN_Data = pd.read_csv('GA_Query_CleanLIWC') 

In [5]:
#Inspect and Clean the Data

UN_Data.head(5)

UN_Data.drop(['Unnamed: 0'], axis = 1, inplace= True)

UN_Data

,date,Class M,Class S,Class I,Class P,Class B,Policy Passed,Conflict Indicator,WC,Analytic,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,2012,1,0,0,0,0,0,1,20075.0,99.00,...,4.34,0.03,0.04,0.00,0.0,1.23,0.07,0.64,0.82,0.60
1,2012,0,0,3,0,0,0,0,822.0,99.00,...,3.04,1.70,0.00,0.00,0.0,0.85,0.49,0.12,4.38,1.46
2,2003,0,0,0,0,0,0,0,314.0,99.00,...,3.50,0.96,0.00,0.00,0.0,2.23,0.00,0.64,3.18,2.87
3,1995,0,0,0,0,0,1,1,17021.0,98.45,...,4.91,0.25,0.17,0.02,0.0,1.33,0.22,0.16,0.64,2.18
4,2007,0,0,0,0,0,0,0,9289.0,98.94,...,3.80,0.16,0.15,0.00,0.0,0.93,0.28,0.75,1.42,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10207,2004,0,0,0,0,0,0,0,4059.0,98.95,...,3.72,0.15,0.12,0.00,0.0,1.18,0.00,0.76,1.72,1.23
10208,1994,0,0,0,0,0,0,0,8210.0,99.00,...,3.58,0.12,0.22,0.00,0.0,1.06,0.02,0.29,1.05,1.75
10209,2013,0,0,0,0,0,0,0,583.0,99.00,...,3.09,0.86,0.00,0.69,0.0,1.89,0.34,0.00,4.80,4.63
10210,2009,0,0,0,0,0,0,0,1562.0,98.88,...,2.82,0.19,0.45,0.00,0.0,1.34,0.00,0.77,1.66,2.18


In [6]:
#Inspect the data by key descriptive statistics

UN_Data.describe()

,date,Class M,Class S,Class I,Class P,Class B,Policy Passed,Conflict Indicator,WC,Analytic,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
count,10212.000000,10212.000000,10212.000000,10212.000000,10212.000000,10212.000000,10212.00000,10212.000000,10190.000000,10190.000000,...,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000,10190.000000
mean,2005.852135,0.032805,0.009205,0.168625,0.051900,0.021641,0.13964,0.457697,9442.204220,98.190282,...,5.193233,0.303105,0.144628,0.031364,0.014043,1.154469,0.260464,0.397916,1.744508,1.768528
std,7.542111,0.229542,0.099521,0.861341,0.307019,0.156540,0.34663,0.498232,7786.325195,1.085791,...,3.290078,0.466117,0.190811,0.105541,0.713727,0.643019,0.439789,1.273977,1.466890,4.337472
min,1993.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,44.000000,80.460000,...,0.210000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.130000,0.010000
25%,1999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3838.500000,97.880000,...,3.980000,0.090000,0.060000,0.000000,0.000000,0.800000,0.050000,0.230000,0.860000,0.740000
50%,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,7439.500000,98.440000,...,4.440000,0.160000,0.110000,0.000000,0.000000,1.040000,0.130000,0.340000,1.330000,1.170000
75%,2012.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,12438.750000,99.000000,...,4.990000,0.320000,0.190000,0.020000,0.000000,1.380000,0.300000,0.490000,2.090000,2.070000
max,2020.000000,9.000000,2.000000,28.000000,9.000000,3.000000,1.00000,1.000000,74776.000000,99.000000,...,47.760000,22.710000,11.150000,3.400000,49.920000,22.880000,14.030000,89.620000,17.900000,213.040000


In [7]:
#Group the data by our label (dependent variable) of policy passage

UN_Data.groupby(['Policy Passed']).count()

,date,Class M,Class S,Class I,Class P,Class B,Conflict Indicator,WC,Analytic,Clout,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
Policy Passed,,,,,,,,,,,,,,,,,,,,,
0,8786,8786,8786,8786,8786,8786,8786,8764,8764,8764,...,8764,8764,8764,8764,8764,8764,8764,8764,8764,8764
1,1426,1426,1426,1426,1426,1426,1426,1426,1426,1426,...,1426,1426,1426,1426,1426,1426,1426,1426,1426,1426


In [8]:
#Normalize the data 

#UN_Data1 = tf.keras.utils.normalize(UN_Data.drop(columns = ['Policy Passed', 'Conflict Indicator']))

#UN_Data1["Policy Passed"] = UN_Data['Policy Passed']

#UN_Data1.describe()

In [9]:
#Divide our variables between the independent variables (features) and dependent variables (policy passage)

labels = UN_Data ['Policy Passed'] + UN_Data['Conflict Indicator']
features = UN_Data.drop(columns= ['Policy Passed', 'Conflict Indicator'])

In [10]:
#Drop Null Values

features = features.fillna(0)
labels = labels.fillna(0)

In [11]:
#Inspect shape of features

features = pd.get_dummies(features)
features.shape[0:5]

(10212, 99)

In [12]:
#Define type of feature and label values

features = features.values.astype('float32')
labels = labels.values.astype('float32')

In [13]:
#Data Sets for Training

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)
features_train, features_validation, labels_train, labels_validation = train_test_split(features_train, labels_train, test_size=0.2)

In [14]:
#Define Precision, Recall, and F1 score metrics
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras

In [15]:
#Create your model

model1 = tf.keras.Sequential([keras.layers.Dense(32, input_shape=(99,)),
                              keras.layers.Dropout(.20),
                              keras.layers.Dense(16, activation= 'relu'),
                              keras.layers.Dropout(.10),
                              keras.layers.Dense(2, activation='softmax')])

In [16]:
### Inspect form of model

tf.keras.utils.plot_model(model1, to_file='model.png', show_shapes = True, show_dtype=True, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [17]:
#Check Trainable Parameters
# Note: All the models are similarly structured

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                3200      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 3,762
Trainable params: 3,762
Non-trainable params: 0
_________________________________________________________________


In [18]:
#Set checkpoints, metrics, loss, and optimizer functions for the model

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc', tf.keras.metrics.Precision(), recall])


In [19]:
#Run the model

print("Model 1 Fitting")
history1 = model1.fit(features_train, labels_train, epochs=30, validation_data=(features_validation, labels_validation), callbacks=[tensorboard_callback])

Model 1 Fitting
Epoch 1/30


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outputs = model.t

In [ ]:
#Run model predictions

# Model 1 
prediction_features_1 = model1.predict(features_test)
performance1 = model1.evaluate(features_test, labels_test)
print(performance1)

In [ ]:
# Averages

# Loss
loss_avg = (performance1[0] + performance2[0] + performance3[0] + performance4[0] + performance5[0] + performance6[0] + performance7[0] + performance8[0] + performance9[0] + performance10[0]
            + performance11[0] + performance12[0] + performance13[0] + performance14[0] + performance15[0] + performance16[0] + performance17[0] + performance18[0] + performance19[0] + performance20[0]
            + performance21[0] + performance22[0] + performance23[0] + performance24[0] + performance25[0] + performance26[0] + performance27[0] + performance28[0] + performance29[0] + performance30[0])/30
print("Loss Average: ", loss_avg)

# Accuracy
acc_avg =(performance1[1] + performance2[1] + performance3[1] + performance4[1] + performance5[1] + performance6[1] + performance7[1] + performance8[1] + performance9[1] + performance10[1]
            + performance11[1] + performance12[1] + performance13[1] + performance14[1] + performance15[1] + performance16[1] + performance17[1] + performance18[1] + performance19[1] + performance20[1]
            + performance21[1] + performance22[1] + performance23[1] + performance24[1] + performance25[1] + performance26[1] + performance27[1] + performance28[1] + performance29[1] + performance30[1])/30
print("Accuraccy Average: ", acc_avg)

# Precision
precision_avg = (performance1[2] + performance2[2] + performance3[2] + performance4[2] + performance5[2] + performance6[2] + performance7[2] + performance8[2] + performance9[2] + performance10[2]
            + performance11[2] + performance12[2] + performance13[2] + performance14[2] + performance15[2] + performance16[2] + performance17[2] + performance18[2] + performance19[2] + performance20[2]
            + performance21[2] + performance22[2] + performance23[2] + performance24[2] + performance25[2] + performance26[2] + performance27[2] + performance28[2] + performance29[2] + performance30[2])/30
print("Precision Average: ", precision_avg)

# Recall
recall_avg = (performance1[3] + performance2[3] + performance3[3] + performance4[3] + performance5[3] + performance6[3] + performance7[3] + performance8[3] + performance9[3] + performance10[3]
            + performance11[3] + performance12[3] + performance13[3] + performance14[3] + performance15[3] + performance16[3] + performance17[3] + performance18[3] + performance19[3] + performance20[3]
            + performance21[3] + performance22[3] + performance23[3] + performance24[3] + performance25[3] + performance26[3] + performance27[3] + performance28[3] + performance29[3] + performance30[3])/30
print("Recall Average: ", recall_avg)

# f1_metric
f1_avg = (performance1[4] + performance2[4] + performance3[4] + performance4[4] + performance5[4] + performance6[4] + performance7[4] + performance8[4] + performance9[4] + performance10[4]
            + performance11[4] + performance12[4] + performance13[4] + performance14[4] + performance15[4] + performance16[4] + performance17[4] + performance18[4] + performance19[4] + performance20[4]
            + performance21[4] + performance22[4] + performance23[4] + performance24[4] + performance25[4] + performance26[4] + performance27[4] + performance28[4] + performance29[4] + performance30[4])/30
print("F1 Average: ", f1_avg)

In [ ]:
#Take the standard deviation of the model samples

#Loss SE
Loss_SE = statistics.stdev([performance1[0], performance2[0], performance3[0], performance4[0], performance5[0],
                  performance6[0], performance7[0], performance8[0], performance9[0], performance10[0],
                  performance11[0], performance12[0], performance13[0], performance14[0], performance15[0],
                  performance16[0], performance17[0], performance18[0], performance19[0], performance20[0], 
                  performance21[0], performance22[0], performance23[0], performance24[0], performance25[0],
                  performance26[0], performance27[0], performance28[0], performprecisionance29[0], performance30[0]])
print("Loss SE:", Loss_SE)

#Accuracy SE
Acc_SE = statistics.stdev([performance1[1], performance2[1], performance3[1], performance4[1], performance5[1],
                  performance6[1], performance7[1], performance8[1], performance9[1], performance10[1],
                  performance11[1], performance12[1], performance13[1], performance14[1], performance15[1],
                  performance16[1], performance17[1], performance18[1], performance19[1], performance20[1], 
                  performance21[1], performance22[1], performance23[1], performance24[1], performance25[1],
                  performance26[1], performance27[1], performance28[1], performance29[1], performance30[1]])
print("Accuraccy SE: ", Acc_SE)

#Precision SE
precision_SE = statistics.stdev([performance1[2], performance2[2], performance3[2], performance4[2], performance5[2],
                  performance6[2], performance7[2], performance8[2], performance9[2], performance10[2],
                  performance11[2], performance12[2], performance13[2], performance14[2], performance15[2],
                  performance16[2], performance17[2], performance18[2], performance19[2], performance20[2], 
                  performance21[2], performance22[2], performance23[2], performance24[2], performance25[2],
                  performance26[2], performance27[2], performance28[2], performance29[2], performance30[2]])
print("Precision SE: ", precision_SE)

#Recall 
Recall_SE = statistics.stdev([performance1[3], performance2[3], performance3[3], performance4[3], performance5[3],
                  performance6[3], performance7[3], performance8[3], performance9[3], performance10[3],
                  performance11[3], performance12[3], performance13[3], performance14[3], performance15[3],
                  performance16[3], performance17[3], performance18[3], performance19[3], performance20[3], 
                  performance21[3], performance22[3], performance23[3], performance24[3], performance25[3],
                  performance26[3], performance27[3], performance28[3], performance29[3], performance30[3]])
print("Recall SE: ", Recall_SE)

#F1 Score
F1_Score_SE = statistics.stdev([performance1[4], performance2[4], performance3[4], performance4[4], performance5[4],
                  performance6[4], performance7[4], performance8[4], performance9[4], performance10[4],
                  performance11[4], performance12[4], performance13[4], performance14[4], performance15[4],
                  performance16[4], performance17[4], performance18[4], performance19[4], performance20[4], 
                  performance21[4], performance22[4], performance23[4], performance24[4], performance25[4],
                  performance26[4], performance27[4], performance28[4], performance29[4], performance30[4]])
print("F1_Score_SE: ", F1_Score_SE)

In [ ]:
#Tensorflow Graphics

%tensorboard --logdir logs/fit